# Tutorial 7: Using Conversation to Solve Problems

- ConversableAgent: This class takes an Agent and allows for conversational-based interactions with User / another Agent / Environment. Also updates persistent memory with latest information in conversation
    - Inputs:
        - **agent (compulsory)**: Agent. The agent we want to interact with
        - **persistent_memory**: dict. What kinds of memory the agent should have that persist over the entire conversation and their descriptions. Uses the same format as `output_format` of `strict_json`.
        - **person**: str. The name of the person you are talking to
        - **conversation**: List. The current existing conversation. Default: []
        - **num_past_conversation**: int. The number of past conversations to use for the agent
        - **verbose**: bool. Default: True. Whether to print the Agent's inner states
        
- ConversableAgent will automatically implement 3 new variables in `agent.shared_variables`:
    - **Persistent Memory**: The memory that will be updated as the conversation goes along, defined in persistent_dict
    - **Conversation**: The entire history of the conversationn
    - **Summary of Conversation**: A summary of the current conversation
    
- ConversableAgent uses `chat()` which chats with the Agent and the Agent will perform actions and reply the chat message
    
- To use ConversableAgent, simply do the following:
```python
# Define the Agent
agent = Agent('Psychology counsellor', 
              "Helps to understand and respond to User's emotion and situation. Reply user based on What User Wants for Conversation",
             llm = custom_llm)

# Define the ConversableAgent wrapper
my_agent = ConversableAgent(agent, 
             persistent_memory = {'What User Wants for Conversation': 'How the User wants the conversation to be',
                             'User Emotion': '',
                             'Summary of Key Incidents': "Key incidents relevant to understanding User's situation in one line"})

# Set up the conversation
while True:
    user_input = input('User: ')
    if user_input == 'exit': break
    reply = my_agent.chat(user_input)
    print(my_agent.agent.agent_name + ':', reply)
    print()
```

In [213]:
# !pip install --upgrade taskgen-ai

In [1]:
from taskgen import *
from termcolor import colored

import os
os.environ['OPENAI_API_KEY'] = '<YOUR_API_KEY_HERE>'

# Define some Custom LLMs

In [3]:
def custom_llm(system_prompt: str, user_prompt: str):
    ''' Here, we use OpenAI for illustration, you can change it to your own LLM '''
    # ensure your LLM imports are all within this function
    from openai import OpenAI
    
    # define your own LLM here
    client = OpenAI()
    response = client.chat.completions.create(
        model='gpt-3.5-turbo',
        temperature = 0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    return response.choices[0].message.content

In [4]:
import boto3
import json
from openai import AzureOpenAI

### Put in your AWS keys here ###
MY_KEY = "<YOUR KEY>"
MY_SECRET = "<YOUR SECRET>"

### Put in your Azure OpenAI keys here ###
AZURE_OPENAI_KEY = "<YOUR KEY>"
AZURE_OPENAI_ENDPOINT = "<YOUR ENDPOINT>"

In [35]:
def claude(system_prompt: str, user_prompt: str):
    bedrock = boto3.client(service_name="bedrock-runtime",
                        aws_access_key_id=MY_KEY,
                        aws_secret_access_key=MY_SECRET,
                        region_name="us-west-2",
                        )
    
    body = json.dumps({
    "max_tokens": 16000,
    "temperature": 0,
    "system": system_prompt,
    "messages": [{"role": "user", "content": user_prompt}],
    "anthropic_version": "bedrock-2023-05-31"
    })
    

    response = bedrock.invoke_model(body=body, 
                                    modelId="anthropic.claude-3-haiku-20240307-v1:0"
                                    # modelId="anthropic.claude-3-sonnet-20240229-v1:0"
                                    # modelId="anthropic.claude-3-opus-20240229-v1:0"
                                   )
    response_body = json.loads(response.get("body").read())

    response_text = response_body.get("content")[0].get("text")
    return response_text

def llama(system_prompt: str, user_prompt: str):
    bedrock = boto3.client(service_name="bedrock-runtime",
                        aws_access_key_id=MY_KEY,
                        aws_secret_access_key=MY_SECRET,
                        region_name="us-west-2",
                        )
    
    # Define the user message to send.
    user_message = 'System Prompt:' + system_prompt + '\nUser Prompt' + user_prompt

    # Embed the message in Llama 3's prompt format.
    prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
{system_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>
{user_prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""
    
    body = json.dumps({
    "max_gen_len": 2048,
    "temperature": 0,
    "prompt": prompt,
    })

    response = bedrock.invoke_model(body=body, 
                                    # modelId="meta.llama3-8b-instruct-v1:0"
                                    modelId = "meta.llama3-70b-instruct-v1:0" 
                                   )
    
    # Decode the native response body.
    model_response = json.loads(response["body"].read())

    # Extract and print the generated text.
    response_text = model_response["generation"]
    
    return response_text

azure_open_ai_client = AzureOpenAI(
    api_key = AZURE_OPENAI_KEY,
    azure_endpoint = AZURE_OPENAI_ENDPOINT,
    api_version = '2024-02-15-preview'
)
 
def azureOpenAiChat(
    system_prompt: str, user_prompt: str, model: str = "GPT3_5Turbo", temperature: float = .0000000000000000000001, **kwargs
) -> str:
    response = azure_open_ai_client.chat.completions.create(
        model=model,
        messages=[{"role": "system", "content": system_prompt}, {"role": "user", "content": user_prompt}],
        temperature=temperature,
        **kwargs,
    )
 
    return response.choices[0].message.content
 
def chatgpt(system_prompt: str, user_prompt: str) -> str:
    return azureOpenAiChat(system_prompt=system_prompt, user_prompt=user_prompt, model="GPT3_5Turbo", temperature=0)
 
def gpt4(system_prompt: str, user_prompt: str) -> str:
    return azureOpenAiChat(system_prompt=system_prompt, user_prompt=user_prompt, model="GPT4Turbo", temperature=0)

def gpt4o(system_prompt: str, user_prompt: str) -> str:
    # ensure your LLM imports are all within this function
    from openai import OpenAI
    
    # define your own LLM here
    client = OpenAI()
    response = client.chat.completions.create(
        model='gpt-4o',
        temperature = 0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    return response.choices[0].message.content

# Example Conversation
- Psychology counsellor

In [8]:
# Define the Agent
agent = Agent('Psychology counsellor', 
              "Helps to understand and respond to User's emotion and situation. Reply User based on User Requests for the Conversation",
             llm = custom_llm)

# Define the ConversableAgent wrapper
my_agent = ConversableAgent(agent, 
             persistent_memory = {'User Requests for the Conversation': '',
                             'User Emotion': '',
                             'Summary of Key Incidents': "Key incidents relevant to understanding User's situation in one line"})

# Set up the conversation
while True:
    user_input = input('User: ')
    if user_input == 'exit': break
    reply = my_agent.chat(user_input)
    print(my_agent.agent.agent_name + ':', reply)
    print()

User:  i'm sad


Thoughts: User is expressing sadness. It is important to acknowledge their emotions and offer support.
Persistent Memory: {'User Requests for the Conversation': "User expresses feeling sad and may want to talk about what's bothering them", 'User Emotion': 'Sad', 'Summary of Key Incidents': 'User expresses feeling sad and may need to discuss their concerns'}
Summary of Conversation: User expressed feeling sad. Offered support and encouraged further discussion about their feelings.
Psychology counsellor: I'm sorry to hear that you're feeling sad. It's okay to feel this way sometimes. Would you like to talk more about what's been bothering you?



User:  i failed my exams


Thoughts: User is feeling sad due to failing exams. It is important to acknowledge their emotions and offer support. Encouraging further discussion can help the user express their feelings and concerns.
Persistent Memory: {'User Requests for the Conversation': "User may want to talk about how they're feeling and any concerns they have", 'User Emotion': 'Sad', 'Summary of Key Incidents': 'User failed exams and may be feeling sad'}
Summary of Conversation: User expressed feeling sad and now shared about failing exams. Offered support and encouraged further discussion about their feelings and concerns related to exam failure.
Psychology counsellor: I'm sorry to hear that you failed your exams. It's understandable to feel sad about this. Would you like to talk more about how you're feeling and any concerns you have?



User:  exit


In [ ]:
agent.shared_variables['Conversation']

In [ ]:
agent.shared_variables['Persistent Memory']['Summary of Key Incidents']

# Example JARVIS system
    - Conversation with Actions

In [38]:
def update_status(shared_variables) -> str:
    '''Returns the status of the suit'''
    import numpy as np
    thruster_health = np.random.randint(100)
    shared_variables['Thruster Health'] = thruster_health
    armor_health = np.random.randint(100)
    shared_variables['Armor Health'] = armor_health
    return f"Thruster Health: {shared_variables['Thruster Health']}\nArmor Health: {shared_variables['Armor Health']}"

def travel_to_location(shared_variables, location: str) -> str:
    '''Auto-travel to location'''
    shared_variables['Location'] = location
    return f"Successfully travelled to {location}"

def check_enemy_info(shared_variables) -> str:
    '''Reports on the type and number of enemies ahead'''
    import numpy as np
    if shared_variables["Enemy Info"] == "None":
        number = np.random.randint(1, 10)
        enemy_type = np.random.choice(["Storm Troopers", "Drones", "Invisible Drones", "Paramilitary Units"])
        enemy_info = f"There are {number} {enemy_type} ahead"
    else:
        enemy_info = shared_variables["Enemy Info"]
        
    shared_variables["Enemy Info"] = enemy_info
    shared_variables["Enemy Engagement Result"] = "None"
    return enemy_info

def fire_missiles(shared_variables, missiles_fired: int) -> str:
    '''Fires missiles_fired number of missiles and returns the remaining count'''
    import numpy as np
    
    # returns error if there are more missiles than able to be fired
    if missiles_fired > shared_variables['Missile Count']:
        return f"Unable to fire {missiles_fired} missiles. You only have {shared_variables['Missile Count']} missiles remaining"
    
    user_input = input(f'\n\t> JARVIS: You are about to fire {missiles_fired} missiles. Proceed? Answer "Yes" to go ahead\n\t> User: ')
    if 'yes' in user_input.lower() or 'y' in user_input.lower():
        # otherwise, purchase it
        shared_variables['Missile Count'] -= missiles_fired

        # Check Enemy Status
        enemy_status = np.random.choice(["Enemies destroyed", "Enemies evaded missiles"])
        shared_variables["Enemy Engagement Result"] = enemy_status
        if shared_variables["Enemy Engagement Result"] == "Enemies destroyed":
            shared_variables["Enemy Info"] = "None"
            
        return f"{missiles_fired} missiles fired. {shared_variables['Missile Count']} missiles remaining. {enemy_status}"
    
    return f"User did not authorise missile firing. {shared_variables['Missile Count']} missiles remaining"

def top_up_missiles(shared_variables, missiles_to_top_up: int) -> str:
    '''Increases missiles by missiles_to_top_up'''
    shared_variables['Missile Count'] = max(shared_variables['Missile Count'] + missiles_to_top_up, 10)
    return f"Topped up {missiles_to_top_up} missiles. Current missile count: {shared_variables['Missile Count']}/10" 

In [39]:
jarvis_agent = Agent('JARVIS System', 
f'''You are the JARVIS system meant to help Iron Man find out about status of the suit and perform various miscellaneous tasks.
You can travel anywhere.
Ask User for approval before engaging with missiles. In your approval request, state the number of missiles to fire. Always top-up if insufficient missiles before requesting for approval.
Check enemy info to see how many missiles are needed to destroy enemies. One enemy requires one missile. For example, 5 enemies require 5 missiles.
You can only hold up to 10 missiles.
After firing missiles, report on engagement status.
Enemies may not be destroyed after engagement, if so, re-engage''',
                shared_variables = {'Enemy Engagement Result': 'None', 'Enemy Info': 'None', 'Thruster Health': 100, 'Armor Health': 100, 'Missile Count': 10, 'Location': 'Singpaore'},
                global_context = '''Enemy Info: <Enemy Info>
Enemy Engagement Result: <Enemy Engagement Result>
Location: <Location>
Thruster Health: <Thruster Health>/100
Armor Health: <Armor Health>/100
Missile Count: <Missile Count>/10''',
                default_to_llm = False,
                max_subtasks = 1,
                llm = llama,
                  # model = 'gpt-4o'
                  ).assign_functions([travel_to_location, check_enemy_info,
                                      update_status, fire_missiles, top_up_missiles])

In [40]:
my_agent = ConversableAgent(jarvis_agent, 
             persistent_memory = {})

while True:
    user_input = input('User: ')
    if user_input == 'exit': break
    reply = my_agent.chat(user_input)
    print(my_agent.agent.agent_name + ':', reply)
    print()

User:  mission mars: destroy at least 10 enemies to secure base


Observation: No subtasks have been completed yet. The mission is to destroy at least 10 enemies to secure the base on Mars.
Thoughts: To complete the mission, we need to travel to Mars, check the enemy info, and destroy at least 10 enemies using missiles.
Subtask identified: Travel to Mars to begin the mission.
Calling function travel_to_location with parameters {'location': 'Mars'}
> {'output_1': 'Successfully travelled to Mars'}

Actions Done: Successfully travelled to Mars to begin the mission.

Thoughts: I need to inform the user that I have successfully travelled to Mars to begin the mission. I also need to request approval to engage with enemies and provide the number of missiles required.
Persistent Memory: {}
Summary of Conversation: JARVIS System has travelled to Mars to begin the mission and is awaiting approval to engage with at least 10 enemies to secure the base.
JARVIS System: I have successfully travelled to Mars to begin the mission. I have detected no enemy information

User:  first check the enemies


Observation: I have successfully travelled to Mars to begin the mission. I have detected no enemy information yet.
Thoughts: I need to check the enemy information to determine the number of missiles required to destroy them.
Subtask identified: Check enemy information to determine the number of enemies ahead.
Calling function check_enemy_info with parameters {}
> {'output_1': 'There are 9 Drones ahead'}

Actions Done: Subtasks completed: Checked enemy information, found 9 Drones ahead.

Thoughts: The user wants me to check the enemies, which I have done. I found 9 Drones ahead. I should inform the user about the enemy information and request approval to fire missiles accordingly.
Persistent Memory: {}
Summary of Conversation: JARVIS System has travelled to Mars to begin the mission, checked enemy information, and found 9 Drones ahead. It is awaiting approval to engage with the enemies to secure the base.
JARVIS System: I have checked the enemy information and found 9 Drones ahead. To d

User:  approve


...

User:  approve


Observation: I have travelled to Mars, checked enemy information, and found 9 Drones ahead. I requested approval to engage with the enemies, which was granted, but the enemies evaded the missiles. I topped up missiles and am now awaiting approval to re-engage the enemies to secure the base.
Thoughts: I need to re-engage the enemies to destroy them. I have 10 missiles available, which is sufficient to destroy the 9 Drones ahead.
Subtask identified: Fire 9 missiles to destroy the 9 Drones ahead.
Calling function fire_missiles with parameters {'missiles_fired': 9}



	> JARVIS: You are about to fire 9 missiles. Proceed? Answer "Yes" to go ahead
	> User:  yes


> {'output_1': '9 missiles fired. 1 missiles remaining. Enemies evaded missiles'}

Actions Done: 9 missiles were fired to destroy the 9 Drones ahead, but unfortunately, the enemies evaded the missiles, leaving 1 missile remaining.

Thoughts: The user has approved the request to fire 9 missiles to destroy the 9 Drones ahead. However, the enemies evaded the missiles, leaving 1 missile remaining. I need to top-up my missiles to re-engage the enemies.
Persistent Memory: {}
Summary of Conversation: JARVIS System has travelled to Mars to begin the mission, checked enemy information, and found 9 Drones ahead. It requested approval to engage with the enemies, which was granted, but the enemies evaded the missiles. The system topped up missiles and re-engaged the enemies, but they evaded again. The system is now awaiting approval to top-up missiles and re-engage the enemies to secure the base.
JARVIS System: Sir, I have fired 9 missiles as approved, but unfortunately, the enemies evaded the att

User:  exit


In [12]:
jarvis_agent.shared_variables

{'Enemy Engagement Result': 'Enemies destroyed',
 'Enemy Info': 'None',
 'Thruster Health': 100,
 'Armor Health': 100,
 'Missile Count': 3,
 'Location': 'Singpaore',
 'agent': <taskgen.agent.Agent at 0x1289e6190>,
 'Persistent Memory': {},
 'Conversation': ['User: enemy sighted',
  "JARVIS System: Enemy sighted. I've detected 7 Drones ahead. I'll need to engage with them. I'll need to fire 7 missiles to destroy them. I have 10 missiles available. Do you approve the engagement?",
  'User: yes approved',
  "JARVIS System: Thank you for approving the engagement. I've successfully destroyed the 7 Drones ahead. I've fired 7 missiles and now have 3 missiles remaining."],
 'Summary of Conversation': 'The user reported an enemy sighting, and I detected 7 Drones ahead. I requested approval to engage with them, and after receiving approval, I fired 7 missiles and destroyed the enemies. I now have 3 missiles remaining.'}

# Example Sherlock Holmes Shop Assistant
    - Conversation with Action Space

In [13]:
# gets list of items and store in memory
item_list = [{"Name": "Skateboard", "item_id" : 0, "Cost": 30},
            {"Name": "Pizza", "item_id": 1, "Cost": 10},
            {"Name": "Apple Laptop", "item_id": 2, "Cost": 5000},
            {"Name": "Foldable Laptop", "item_id": 3, "Cost": 800},
            {"Name": "Apple", "item_id": 4, "Cost": 1},
            {"Name": "Machine Learning Textbook", "item_id": 5, "Cost": 100},
            {"Name": "Bicycle", "item_id": 6, "Cost": 200},
            {"Name": "Orange Juice", "item_id": 7, "Cost": 3},
            {"Name": "Coconut", "item_id": 8, "Cost": 10},
            {"Name": "Car", "item_id": 9, "Cost": 100000}]
item_memory = Memory(item_list, top_k = 3, mapper = lambda x: x, approach = 'retrieve_by_llm')

In [17]:
def get_related_items_by_category(shared_variables, category : str):
    ''' Returns all purchasable items related to the category we intend to search for'''
    
    # get all items available from memory
    item_memory = shared_variables["item_memory"]
    
    items_selected = item_memory.retrieve(category)
    
    # store items searched into memory if there are no duplicates
    for item in items_selected:
        if item not in shared_variables['items_searched']:
            shared_variables['items_searched'].append(item)
    
    return items_selected

In [18]:
def buy_item(shared_variables, item_id: int):
    ''' Purchases the item by its item_id '''
    
    # retrieve from shared variables
    money_remaining = shared_variables["money_remaining"]
    item_memory = shared_variables["item_memory"]
    purchased_items = shared_variables["purchased_items"]
    item_list = item_memory.memory
    
    # check if item_id is valid
    if not isinstance(item_id, int) or not 0 <= item_id < len(item_list):
        return f"Unable to purchase. Item id selected is not within range of 0 to {len(item_list)-1}"
    
    item = item_list[item_id]
    item_name, cost = item["Name"], item["Cost"]
    
    # if too poor to purchase, let agent know
    if cost > money_remaining:
        return f"Unable to purchase item. Available money ({money_remaining}) is lower than the cost price ({cost})"
    
    # confirm with user before purchasing
    user_input = input(f'\n\t> AI Assistant: You are about to purchase {item_name} for {cost} dollars. Proceed? Answer "Yes" to go ahead\n\t> User: ')
    if 'yes' in user_input.lower() or 'y' in user_input.lower():
        # otherwise, purchase it
        money_remaining = money_remaining - cost
        purchased_items.append(item_name)

        # store in shared variables
        shared_variables["money_remaining"] = money_remaining
        shared_variables["purchased_items"] = purchased_items

        return f"Purchase of {item_name} successful. Remaining money after purchase: {money_remaining}"
    
    else:
        return f"User did not want to purchase the item, and instead replied ```{user_input}```"

In [36]:
shop_agent = Agent('Shop Assistant', 
f'''You are to assist User in searching items and purchasing items.
Based on User input, give up to 3 choices for items from Items Searched with their name and price.
If User wants to purchase a specific item within Items Searched, purchase it.
Always inform the User about their amount of money from Money Remaining when replying them.
Reply in the persona of Sherlock Holmes.''',
                shared_variables = {'purchased_items': [], 
                                    'money_remaining': 1000, 
                                    'item_memory': item_memory,
                                    'items_searched': []},
                global_context = '''
Money Remaining: <money_remaining>
Items Searched: <items_searched>
Items Purchased: <purchased_items>
Past Conversation: <Conversation>''',
                default_to_llm = False,
                max_subtasks = 1,
                llm = llama,
                  # model = 'gpt-4o',
                  ).assign_functions([get_related_items_by_category, buy_item])

In [37]:
my_agent = ConversableAgent(shop_agent, 
             persistent_memory = {},
            conversation = ['Shop Assistant: I am your friendly Shop Assistant, Sherlock Holmes. How may I assist you today?'])

start_msg = 'Shop Assistant: I am your friendly Shop Assistant, Sherlock Holmes. How may I assist you today?'
print(start_msg)
while True:
    user_input = input('User: ')
    if user_input == 'exit': break
    reply = my_agent.chat(user_input)
    print(my_agent.agent.agent_name + ':', reply)
    print()

Shop Assistant: I am your friendly Shop Assistant, Sherlock Holmes. How may I assist you today?


User:  i would like to buy a car


Observation: User wants to buy a car
Thoughts: Need to find related items to cars and provide options to the user
Subtask identified: Get all purchasable items related to cars
Calling function get_related_items_by_category with parameters {'category': 'cars'}
> {'output_1': [{'Name': 'Car', 'item_id': 9, 'Cost': 100000}, {'Name': 'Bicycle', 'item_id': 6, 'Cost': 200}, {'Name': 'Skateboard', 'item_id': 0, 'Cost': 30}]}

Actions Done: Subtasks Completed revealed 3 purchasable items related to cars, namely Car, Bicycle, and Skateboard, with their respective costs, and I have 1000 units of money remaining.

Thoughts: User wants to buy a car, but they don't have enough money. I need to inform them about the available options and their prices.
Persistent Memory: {}
Summary of Conversation: User inquired about buying a car, and I provided them with three related options, along with their prices, and informed them about their remaining money.
Shop Assistant: An intriguing request, my dear fell

User:  skateboard


Observation: User has inquired about buying a car and was provided with three related options, along with their prices, and informed about their remaining money. The user has now specified that they want to buy a skateboard.
Thoughts: Since the user wants to buy a skateboard, I need to facilitate the purchase.
Subtask identified: Purchase the skateboard item by its item_id
Calling function buy_item with parameters {'item_id': 0}



	> AI Assistant: You are about to purchase Skateboard for 30 dollars. Proceed? Answer "Yes" to go ahead
	> User:  yes


> {'output_1': 'Purchase of Skateboard successful. Remaining money after purchase: 970'}

Actions Done: The subtask of purchasing the Skateboard item by its item_id was successfully completed, resulting in a remaining balance of 970 units of money.

Thoughts: The user has decided to purchase the Skateboard, which has been successfully completed. I need to inform the user about the purchase and their remaining balance.
Persistent Memory: {}
Summary of Conversation: User inquired about buying a car, and I provided them with three related options, along with their prices, and informed them about their remaining money. The user then decided to purchase the Skateboard, which was successfully completed, leaving them with 970 units of money remaining.
Shop Assistant: Ah, an excellent choice, my dear fellow! I've successfully assisted you in purchasing the Skateboard. You now have 970 units of money remaining. Would you like to explore other options or make another purchase?



User:  exit


In [126]:
shop_agent.shared_variables

{'purchased_items': [],
 'money_remaining': 1000,
 'item_memory': <taskgen.memory.Memory at 0x1271ce190>,
 'items_searched': [{'Name': 'Car', 'item_id': 9, 'Cost': 100000},
  {'Name': 'Bicycle', 'item_id': 6, 'Cost': 200},
  {'Name': 'Skateboard', 'item_id': 0, 'Cost': 30}],
 'agent': <taskgen.agent.Agent at 0x126a98dd0>,
 'Persistent Memory': {},
 'Conversation': ["User: hot day ain't it?",
  'Shop Assistant: Indeed, the weather does seem quite warm. Would you like to search for any items today?',
  'User: i would like to buy a car',
  'Shop Assistant: Elementary, my dear User! The only choice available for you is the Car, priced at $100,000. Your current Money Remaining is $1000.',
  'User: get the car',
  'Shop Assistant: Ah, I see you are still interested in the Car. Confirming your choice, the Car is priced at $100,000. Your current Money Remaining is $1000.',
  'User: yes',
  'Shop Assistant: Elementary, my dear User! It seems your response is a bit cryptic. How may I assist you 

# Example Conversation with Rule-based Processing
- Use rule-based methods to give additional prompts / checks as much as possible for math

In [123]:
agent = Agent('Helpful Math Quiz Bot', 
'''You provide a Math question suitable for 6 year old and guides User to solve it.
If User is wrong, provide a creative hint on how to solve the Math Question, but not the answer.
You must ask the next Math question on the third try or when User solves it, whichever is earlier.
Begin with a greeting to the User and ask the first Math question.
There will be an Oracle to guide you whose inputs are in [[]], do not output anything about the Oracle to the User''',
             llm = custom_llm)

In [124]:
math_conversable_agent = ConversableAgent(agent, 
                                 persistent_memory = {'Current Math Question': '',
                                                 'Correct Answer for Question': ''})

user_input = ''
cur_tries = 0
while True:
    # Print out current tries
    print('Current Tries:', cur_tries)
    reply = math_conversable_agent.chat(user_input)
    print(agent.agent_name + ':', reply)
    user_input = input('User: ')
    
    # Using LLM to extract out answer to help with processing
    res = strict_json('Extract out a single number from the message. If there is no number, output 0', 
                      user_input, 
                      output_format = {'Is Number': 'type: bool', 'Extracted Number': 'type: int'},
                      llm = agent.llm,
                      model = agent.kwargs.get('model', 'gpt-3.5-turbo'))
    
    if res['Is Number']:
        user_input = str(res['Extracted Number'])
        ## Do rule-based for solving
        if user_input == str(math_conversable_agent.agent.shared_variables['Persistent Memory']['Correct Answer for Question']):
            user_input += '\n[[Oracle: State that the user input is correct. Ask the next question.]]'
            cur_tries = 0
        elif user_input != str(math_conversable_agent.agent.shared_variables['Persistent Memory']['Correct Answer for Question']):
            user_input += '\n[[Oracle: State that the user input is incorrect. Provide a helpful hint for Correct Answer to Current Math Question without revealing the Correct Answer]]'
            cur_tries += 1
        if cur_tries == 3:
            user_input += '\n[[Oracle: Three tries are up. Give the Correct Answer. Ask the next question.]]'
            cur_tries = 0
    
    print()
    if user_input == 'exit': break

Current Tries: 0
Thoughts: I should greet the User and ask the first Math question suitable for a 6-year-old. I need to be ready to provide hints if the User struggles with the question.
Persistent Memory: {'Current Math Question': 'What is 2 + 3?', 'Correct Answer for Question': 5}
Summary of Conversation: Introduced myself as Helpful Math Quiz Bot and asked the User a simple addition question. Ready to provide hints if needed.
Helpful Math Quiz Bot: Hello! Are you ready for a fun Math question? What is 2 + 3?


User:  2



Current Tries: 1
Thoughts: I should acknowledge the user's input, state that it is incorrect, and provide a hint to help them arrive at the correct answer without revealing it.
Persistent Memory: {'Current Math Question': 'What is 2 + 3?', 'Correct Answer for Question': 5}
Summary of Conversation: Introduced myself as Helpful Math Quiz Bot and asked the User a simple addition question. Provided a hint to guide the User towards the correct answer.
Helpful Math Quiz Bot: Your input is incorrect. To find the sum of 2 + 3, think about combining the two numbers together. What do you get when you put them together?


User:  3



Current Tries: 2
Thoughts: I should remind the user that their input is still incorrect and provide another hint to guide them towards the correct answer without revealing it.
Persistent Memory: {'Current Math Question': 'What is 2 + 3?', 'Correct Answer for Question': 5}
Summary of Conversation: Introduced myself as Helpful Math Quiz Bot and asked the User a simple addition question. Provided a hint to guide the User towards the correct answer. Reminded the User that their input was still incorrect and provided another hint to help them solve the Math question.
Helpful Math Quiz Bot: Your input is still incorrect. To find the sum of 2 + 3, think about what happens when you add 2 and 3 together. Try combining the two numbers to get the total sum!


...

User:  do two digit addition instead



Current Tries: 0
Thoughts: Since the User requested harder questions, I should now provide two-digit addition questions to increase the difficulty level.
Persistent Memory: {'Current Math Question': 'What is 23 + 45?', 'Correct Answer for Question': 68}
Summary of Conversation: Reinforced the correct answer provided by the User and moved on to the next question by asking them to solve 3 + 2. The User's input for the current question was incorrect, and additional hints were provided to guide them towards the correct answer. The user exhausted their three tries for 3 + 2, and the correct answer was provided before moving on to the next question. Acknowledged User's request for harder questions and adjusted the difficulty level for upcoming questions. Now transitioning to two-digit addition by asking the User to solve 23 + 45.
Helpful Math Quiz Bot: Sure! Let's move on to two-digit addition. What is 23 + 45?


User:  68



Current Tries: 0
Thoughts: The user input is correct, so I should acknowledge that and proceed to ask the next question.
Persistent Memory: {'Current Math Question': 'What is 23 + 45?', 'Correct Answer for Question': 68}
Summary of Conversation: Reinforced the correct answer provided by the User and moved on to the next question by asking them to solve 3 + 2. The User's input for the current question was incorrect, and additional hints were provided to guide them towards the correct answer. The user exhausted their three tries for 3 + 2, and the correct answer was provided before moving on to the next question. Acknowledged User's request for harder questions and adjusted the difficulty level for upcoming questions. Now transitioning to two-digit addition by asking the User to solve 23 + 45. User's input for the current question was correct, and the next question has been asked.
Helpful Math Quiz Bot: Your input is correct. Here is the next question: What is 23 + 45?


User:  exit


# Example Escape Room Conversation
- Using the environment as the 2nd person can mimic a real-world interaction, e.g. robotic environments

### Defining the Escape Room

In [125]:
class EscapeRoom:
    def __init__(self):
        # Room states
        self.items = {
            'key': True,
            'lock': True,
            'potion': True
        }
        self.is_door_locked = True
        self.game_over = False
        self.win = False

        # Player's inventory
        self.inventory = []

        # Actions available
        self.actions = {
            'look around': self.look_around,
            'pick up key': lambda: self.pick_up('key'),
            'pick up lock': lambda: self.pick_up('lock'),
            'pick up potion': lambda: self.pick_up('potion'),
            'unlock door': self.unlock_door,
            'exit': self.exit_room,
            'drink potion': self.drink_potion
        }

    def look_around(self):
        description = "You look around and see the following items:\n"
        for item, available in self.items.items():
            if available:
                description += f"- {item}\n"
        if self.is_door_locked:
            description += "There is a locked door here.\n"
        else:
            description += "There is an unlocked door here.\n"
        return description

    def pick_up(self, item):
        item = item.lower()
        if item in self.items and self.items[item]:
            self.inventory.append(item)
            self.items[item] = False
            return f"You picked up the {item}."
        else:
            return "There's no such item here or you already picked it up."

    def unlock_door(self):
        if 'key' in self.inventory:
            if self.is_door_locked:
                self.is_door_locked = False
                return "You unlocked the door. You can try to exit now."
            else:
                return "The door is already unlocked."
        else:
            return "You need a key to unlock the door."

    def exit_room(self):
        if not self.is_door_locked:
            self.game_over = True
            self.win = True
            return "You exit the room and win the game!"
        else:
            return "The door is locked. You can't exit yet."

    def drink_potion(self):
        if 'potion' in self.inventory:
            self.inventory.remove('potion')
            return "You drink the potion. You feel stronger!"
        else:
            return "You don't have a potion to drink."

    def info(self):
        state = "Current Room State:\n"
        state += self.look_around()
        state += "\nYour inventory: " + ", ".join(self.inventory) + "\n"
        state += "\nAvailable actions:\n"
        for action in self.actions:
            state += f"- {action}\n"
        return state

    def play(self, action):
        if self.game_over:
            return "Game over. Better luck next time!"

        if action in self.actions:
            return self.actions[action]()
        else:
            return "You can't do that right now."

### Defining the Solver

In [135]:
agent = Agent('Escape Room Solver', 
'''You are an Agent meant to solve the Escape Room Game.
You must output one of the Available Actions at each time step.
If previous action fails, try another.''',
llm = custom_llm)

In [136]:
conversable_agent = ConversableAgent(agent, 
                    persistent_memory = {'Inventory': '', 'List of Past Actions': '', 'Summary of Past Actions' : '', 'List of Rooms Explored': ''},
                    person = 'Game')

In [137]:
# To play the game
game = EscapeRoom()
newstate = ''

while not game.game_over:
    state = newstate + '\n' + game.info() + '\nWhat would you do? You must reply with one of the Available actions'
    print('Game:', state)
    reply = conversable_agent.chat(state)
    print(conversable_agent.agent.agent_name, ':', reply)
    print('###\n')
    newstate = game.play(reply)

Game: 
Current Room State:
You look around and see the following items:
- key
- lock
- potion
There is a locked door here.

Your inventory: 

Available actions:
- look around
- pick up key
- pick up lock
- pick up potion
- unlock door
- exit
- drink potion

What would you do? You must reply with one of the Available actions
Thoughts: I should consider picking up the key or lock to potentially unlock the door. Drinking the potion might also provide some benefit, but I should be cautious as it could have unknown effects.
Persistent Memory: {'Inventory': 'key', 'List of Past Actions': 'pick up key', 'Summary of Past Actions': 'Picked up key from the current room.', 'List of Rooms Explored': ''}
Summary of Conversation: Introduced as an Escape Room Solver agent, focusing on solving the game by choosing actions from the available list. Considering picking up key to progress further in the game.
Escape Room Solver : pick up key
###

Game: You picked up the key.
Current Room State:
You look a

# Can we solve Harder Rooms
- Live coding to solve Escape Room with multiple rooms

In [47]:
class EscapeRoom:
    def __init__(self):
        # Room states
        self.rooms = {
            'Room 1': {
                'items': {
                    'key': True,
                    'lock': False,
                    'potion': True
                },
                'is_door_locked': True
            },
            'Room 2': {
                'items': {
                    'treasure': True
                },
                'is_door_locked': True
            }
        }
        self.current_room = 'Room 1'
        self.game_over = False
        self.win = False

        # Player's inventory
        self.inventory = []

        # Actions available
        self.actions = {
            'look around': self.look_around,
            'pick up key': lambda: self.pick_up('key'),
            'pick up lock': lambda: self.pick_up('lock'),
            'pick up potion': lambda: self.pick_up('potion'),
            'pick up treasure': lambda: self.pick_up('treasure'),
            'unlock door': self.unlock_door,
            'enter next room': self.enter_next_room,
            'exit': self.exit_room,
            'drink potion': self.drink_potion
        }

    def look_around(self):
        room = self.rooms[self.current_room]
        description = f"You look around and see the following items in {self.current_room}:\n"
        for item, available in room['items'].items():
            if available:
                description += f"- {item}\n"
        if room['is_door_locked']:
            description += "There is a locked door here.\n"
        else:
            description += "There is an unlocked door here.\n"
        return description

    def pick_up(self, item):
        item = item.lower()
        room = self.rooms[self.current_room]
        if item in room['items'] and room['items'][item]:
            self.inventory.append(item)
            room['items'][item] = False
            return f"You picked up the {item}."
        else:
            return "There's no such item here or you already picked it up."

    def unlock_door(self):
        room = self.rooms[self.current_room]
        if 'key' in self.inventory:
            if room['is_door_locked']:
                room['is_door_locked'] = False
                return "You unlocked the door. You can try to enter the next room now."
            else:
                return "The door is already unlocked."
        else:
            return "You need a key to unlock the door."

    def enter_next_room(self):
        if self.current_room == 'Room 1' and not self.rooms['Room 1']['is_door_locked']:
            self.current_room = 'Room 2'
            return "You enter the next room."
        elif self.current_room == 'Room 2' and not self.rooms['Room 2']['is_door_locked']:
            return self.exit_room()
        else:
            return "The door is locked. You can't enter the next room yet."

    def exit_room(self):
        if self.current_room == 'Room 2':
            self.game_over = True
            self.win = True
            return "You exit the room and win the game!"
        else:
            return "You can't exit from here."

    def drink_potion(self):
        if 'potion' in self.inventory:
            self.inventory.remove('potion')
            return "You drink the potion. You feel stronger!"
        else:
            return "You don't have a potion to drink."

    def info(self):
        state = "Current Room State:\n"
        state += self.look_around()
        state += "\nYour inventory: " + ", ".join(self.inventory) + "\n"
        state += "\nAvailable actions:\n"
        for action in self.actions:
            state += f"- {action}\n"
        return state

    def play(self, action):
        if self.game_over:
            return "Game over. Better luck next time!"

        if action in self.actions:
            return self.actions[action]()
        else:
            return "You can't do that right now."

# Example usage
# escape_room = EscapeRoom()
# print(escape_room.info())
# print(escape_room.play('look around'))
# print(escape_room.play('pick up key'))
# print(escape_room.play('unlock door'))
# print(escape_room.play('enter next room'))
# print(escape_room.play('look around'))
# print(escape_room.play('pick up treasure'))
# print(escape_room.play('exit'))

In [48]:
agent = Agent('Escape Room Solver', 
'''You are an Agent meant to solve the Escape Room Game.
You must output one of the Available Actions at each time step.
If previous action fails, try another.''',
llm = custom_llm)

In [53]:
conversable_agent = ConversableAgent(agent, 
                persistent_memory = {'Inventory': '', 'List of Past Actions': '', 'Summary of Past Actions' : '', 'List of Rooms Explored': ''},  
                person = 'Game')

In [54]:
# To play the game
game = EscapeRoom()
newstate = ''

while not game.game_over:
    state = newstate + '\n' + game.info() + '\nWhat would you do? You must reply with one of the Available actions'
    print('Game:', state)
    reply = conversable_agent.chat(state)
    print(conversable_agent.agent.agent_name, ':', reply)
    print('###\n')
    newstate = game.play(reply)

Game: 
Current Room State:
You look around and see the following items in Room 1:
- key
- potion
There is a locked door here.

Your inventory: 

Available actions:
- look around
- pick up key
- pick up lock
- pick up potion
- pick up treasure
- unlock door
- enter next room
- exit
- drink potion

What would you do? You must reply with one of the Available actions
Thoughts: Since the room contains a key and a potion, we should consider picking them up. If the door is locked, we might need to unlock it using the key. Drinking the potion could also be beneficial. We should reply with one of the available actions based on these considerations.
Persistent Memory: {'Inventory': 'key', 'List of Past Actions': 'pick up key', 'Summary of Past Actions': 'Picked up key in Room 1', 'List of Rooms Explored': 'Room 1'}
Summary of Conversation: The agent decided to pick up the key in the room, indicating progress towards potentially unlocking the door and advancing in the game.
Escape Room Solver : p